<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=883bcdabee98ce460acb749cad5ee97dc2efd453dd3c6a74009d9ab47f8fd798
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-24 10:19:03
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.42 C
-------------------
Today PnL: -40.31 K (-0.28%)
Current PnL: -19.91 L (-13.26%)
CY Booked + Current PnL: -7.81 L (-5.2%)
-------------------
Total profit:  2.13 L
Total loss:  -22.04 L
-------------------
Total Booked + Current PnL: 19.17 L (15.57%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.54%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.09 L (61.44%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.29%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VOLTAS,1530.0,3.12,60.0,X-MC,2.80,216330.0,24588.0,13174.0,0.77,12.82,6.09,19.69,99.0,1.87,1.53,20.49,XY25,NTT,AC
58,RELIANCE,1533.0,-9.91,65.0,X-LC,3.16,225701.0,15335.0,13452.0,-0.11,7.29,5.96,13.68,37.0,1.14,1.60,25.02,XY25,NTT,REFINERIES
2,ABBOTINDIA,35195.0,-9.24,43.0,X-MC,3.33,89700.0,-978.0,15886.0,-0.71,-1.08,17.71,16.44,101.0,-0.06,0.64,18.88,X40,ATH,PHARMA
78,TTKPRESTIG,770.0,95.09,47.0,M-SC,1.98,84076.0,-16701.0,16798.0,-0.01,-16.57,19.98,0.09,245.0,-0.99,0.60,10.46,OX40N,NTT,DURABLES
42,ITC,452.0,-37.08,61.0,X-LC,1.98,202690.0,2552.0,18343.0,-0.35,1.28,9.05,10.44,4.0,0.14,1.44,6.91,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,31.71,56.0,M-LC,11.08,214738.0,9662.0,102902.0,2.39,4.71,47.92,54.89,52.0,0.09,1.52,29.94,X5K,ATH,METALS
61,SAIL,228.00,42.74,50.0,M-MC,10.05,225582.0,620.0,167043.0,1.28,0.28,74.05,74.53,192.0,0.00,1.60,32.62,XY24,BTT,STEEL
73,TATAELXSI,9161.00,-22.37,55.0,H-MC,7.49,105127.0,-26708.0,68932.0,1.13,-20.26,65.57,32.03,98.0,-0.39,0.74,17.75,OX40N,NTT,IT
36,IEX,219.00,-30.62,61.0,H-SC,17.44,198258.0,-1696.0,98931.0,1.09,-0.85,49.90,48.63,115.0,-0.02,1.40,10.61,XR,NTT,MISC
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COLPAL,3767.14,-5.42,42.0,X-MC,8.17,220600.0,-42765.0,156119.0,-3.54,-16.24,70.77,43.04,84.0,-0.27,1.56,2.41,XY25,ATH,FMCG
31,HINDUNILVR,2922.00,-10.67,43.0,X-LC,7.66,251000.0,-6063.0,41189.0,-3.52,-2.36,16.41,13.67,24.0,-0.15,1.78,17.57,XY25,NTT,FMCG
22,DABUR,735.00,-10.32,47.0,X-MC,3.91,242742.0,-5326.0,113725.0,-2.13,-2.15,46.85,43.70,102.0,-0.05,1.72,11.99,XY24,BTT,FMCG
26,GREENPANEL,537.00,232.04,54.0,M-SC,5.40,148108.0,-34970.0,113954.0,-1.33,-19.10,76.94,43.14,230.0,-0.31,1.05,39.32,XY24,NTT,MISC
72,TANLA,1963.11,-29.77,32.0,H-SC,16.13,188006.0,-82755.0,412711.0,-1.29,-30.56,219.52,121.86,133.0,-0.20,1.33,47.07,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,42.74,50.0,M-MC,10.05,225582.0,620.0,167043.0,1.28,0.28,74.05,74.53,192.0,0.00,1.60,32.62,XY24,BTT,STEEL
37,INDIAMART,4810.62,-55.39,42.0,H-SC,7.15,124555.0,1219.0,130409.0,0.01,0.99,104.70,106.72,119.0,0.01,0.88,24.60,AR,ATH,MISC
36,IEX,219.00,-30.62,61.0,H-SC,17.44,198258.0,-1696.0,98931.0,1.09,-0.85,49.90,48.63,115.0,-0.02,1.40,10.61,XR,NTT,MISC
87,WIPRO,420.00,-13.79,48.0,M-LC,5.91,151609.0,664.0,108795.0,0.09,0.44,71.76,72.51,53.0,0.01,1.07,6.49,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,121.70,68.0,H-SC,15.24,140103.0,-2376.0,21898.0,-0.02,-1.67,15.63,13.70,163.0,-0.11,0.99,65.36,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.87,59.0,H-SC,2.37,228177.0,-41490.0,77831.0,-0.74,-15.39,34.11,13.47,138.0,-0.53,1.62,16.02,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,95.09,47.0,M-SC,1.98,84076.0,-16701.0,16798.0,-0.01,-16.57,19.98,0.09,245.0,-0.99,0.60,10.46,OX40N,NTT,DURABLES
73,TATAELXSI,9161.0,-22.37,55.0,H-MC,7.49,105127.0,-26708.0,68932.0,1.13,-20.26,65.57,32.03,98.0,-0.39,0.74,17.75,OX40N,NTT,IT
18,CERA,9475.0,-23.87,42.0,H-SC,2.64,138736.0,-37167.0,79191.0,0.17,-21.13,57.08,23.89,149.0,-0.47,0.98,19.82,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,47.0,M-LC,5.70,205205.0,2035.0,40097.0,-0.48,1.00,19.54,20.74,55.0,0.05,1.45,6.60,XY25,NTT,FINANCE
61,SAIL,228.00,42.74,50.0,M-MC,10.05,225582.0,620.0,167043.0,1.28,0.28,74.05,74.53,192.0,0.00,1.60,32.62,XY24,BTT,STEEL
32,HINDZINC,730.22,31.71,56.0,M-LC,11.08,214738.0,9662.0,102902.0,2.39,4.71,47.92,54.89,52.0,0.09,1.52,29.94,X5K,ATH,METALS
46,KPIGREEN,731.05,7.10,67.0,H-SC,5.19,127764.0,2467.0,56459.0,-0.31,1.97,44.19,47.03,141.0,0.04,0.90,59.97,MH,ATH,POWER
80,UNIONBANK,163.00,-10.08,61.0,M-LC,9.14,162477.0,21637.0,22861.0,0.03,15.36,14.07,31.59,66.0,0.95,1.15,45.92,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-19.25,70.0,H-MC,14.29,207061.0,27253.0,85164.0,-0.66,15.16,41.13,62.52,88.0,0.32,1.47,54.41,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-38.52,65.0,M-SC,3.83,103912.0,12414.0,66337.0,-0.99,13.57,63.84,86.07,223.0,0.19,0.74,50.90,XR,NTT,DURABLES
32,HINDZINC,730.22,31.71,56.0,M-LC,11.08,214738.0,9662.0,102902.0,2.39,4.71,47.92,54.89,52.0,0.09,1.52,29.94,X5K,ATH,METALS
46,KPIGREEN,731.05,7.10,67.0,H-SC,5.19,127764.0,2467.0,56459.0,-0.31,1.97,44.19,47.03,141.0,0.04,0.90,59.97,MH,ATH,POWER
87,WIPRO,420.00,-13.79,48.0,M-LC,5.91,151609.0,664.0,108795.0,0.09,0.44,71.76,72.51,53.0,0.01,1.07,6.49,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO
24,DMART,5391.45,-16.82,35.0,X-LC,10.17,101765.0,-821.0,27629.0,-0.65,-0.80,27.15,26.13,38.0,-0.03,0.72,24.57,X40N,ATH,FMCG
53,QUESS,986.00,-48.21,35.0,X-SC,38.75,51476.0,-13530.0,164461.0,0.02,-20.81,319.49,232.18,198.0,-0.08,0.36,0.41,XY24,NTT,MISC
21,COLPAL,3767.14,-5.42,42.0,X-MC,8.17,220600.0,-42765.0,156119.0,-3.54,-16.24,70.77,43.04,84.0,-0.27,1.56,2.41,XY25,ATH,FMCG
52,PGHH,17907.65,-29.49,42.0,X-MC,3.22,204045.0,3225.0,64560.0,-0.74,1.61,31.64,33.76,80.0,0.05,1.45,6.89,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-1.97,50.0,H-LC,0.80,157600.0,-28495.0,75979.0,0.07,-15.31,48.21,25.51,15.0,-0.38,1.12,16.14,AR,ATH,ELECTRICAL
42,ITC,452.00,-37.08,61.0,X-LC,1.98,202690.0,2552.0,18343.0,-0.35,1.28,9.05,10.44,4.0,0.14,1.44,6.91,X40,NTT,FMCG
85,VOLTAS,1530.00,3.12,60.0,X-MC,2.80,216330.0,24588.0,13174.0,0.77,12.82,6.09,19.69,99.0,1.87,1.53,20.49,XY25,NTT,AC
29,HAVELLS,2069.16,-5.19,50.0,X-MC,2.86,235201.0,-12530.0,87589.0,0.76,-5.06,37.24,30.30,92.0,-0.14,1.67,7.56,X40,ATH,ELECTRICAL
58,RELIANCE,1533.00,-9.91,65.0,X-LC,3.16,225701.0,15335.0,13452.0,-0.11,7.29,5.96,13.68,37.0,1.14,1.60,25.02,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,35.0,X-SC,38.75,51476.0,-13530.0,164461.0,0.02,-20.81,319.49,232.18,198.0,-0.08,0.36,0.41,XY24,NTT,MISC
21,COLPAL,3767.14,-5.42,42.0,X-MC,8.17,220600.0,-42765.0,156119.0,-3.54,-16.24,70.77,43.04,84.0,-0.27,1.56,2.41,XY25,ATH,FMCG
3,ACC,3906.00,-38.07,48.0,X-MC,2.90,184600.0,-53351.0,205995.0,-0.52,-22.42,111.59,64.15,174.0,-0.26,1.31,3.55,XY24,BTT,CEMENT
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO
10,BAJAJHFL,181.50,-14.27,47.0,X-MC,5.75,180646.0,-19860.0,117022.0,-0.05,-9.91,64.78,48.45,90.0,-0.17,1.28,4.17,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,35.0,X-SC,38.75,51476.0,-13530.0,164461.0,0.02,-20.81,319.49,232.18,198.0,-0.08,0.36,0.41,XY24,NTT,MISC
51,PAGEIND,51769.93,-26.57,44.0,X-MC,6.74,83000.0,-510.0,20542.0,0.10,-0.61,24.75,23.98,82.0,-0.02,0.59,5.17,X40,ATH,APPARELS
57,RELAXO,1176.00,-39.20,46.0,X-SC,4.67,82889.0,-62271.0,138201.0,-0.64,-42.90,166.73,52.31,136.0,-0.45,0.59,10.95,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.24,43.0,X-MC,3.33,89700.0,-978.0,15886.0,-0.71,-1.08,17.71,16.44,101.0,-0.06,0.64,18.88,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-35.15,48.0,X-SC,4.42,97052.0,-31618.0,76914.0,0.15,-24.57,79.25,35.20,219.0,-0.41,0.69,11.39,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-27.40,58.0,X-LC,12.34,288984.0,-56972.0,123685.0,0.04,-16.47,42.80,19.28,1.0,-0.46,2.05,6.83,X40,ATH,IT
40,INFY,2275.00,-16.73,59.0,X-LC,7.17,324910.0,12022.0,159661.0,-0.20,3.84,49.14,54.87,3.0,0.08,2.30,10.75,X40,BTT,IT
42,ITC,452.00,-37.08,61.0,X-LC,1.98,202690.0,2552.0,18343.0,-0.35,1.28,9.05,10.44,4.0,0.14,1.44,6.91,X40,NTT,FMCG
84,VBL,671.64,-18.08,54.0,X-LC,9.86,294884.0,-20958.0,133612.0,-0.72,-6.64,45.31,35.67,5.0,-0.16,2.09,6.30,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,48.0,H-LC,6.32,248904.0,-12715.0,131919.0,0.08,-4.86,53.00,45.57,7.0,-0.10,1.76,5.91,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7172.22,67.0,L-SC,7.22,79803.0,-14007.0,90624.0,-1.03,-14.93,113.56,81.67,269.0,-0.15,0.57,54.54,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,26.97,81024.0,-32525.0,72573.0,0.29,-28.64,89.57,35.27,268.0,-0.45,0.57,98.31,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,48.0,H-SC,9.55,91455.0,-6525.0,28132.0,-0.73,-6.66,30.76,22.05,152.0,-0.23,0.65,32.94,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-38.52,65.0,M-SC,3.83,103912.0,12414.0,66337.0,-0.99,13.57,63.84,86.07,223.0,0.19,0.74,50.90,XR,NTT,DURABLES
46,KPIGREEN,731.05,7.10,67.0,H-SC,5.19,127764.0,2467.0,56459.0,-0.31,1.97,44.19,47.03,141.0,0.04,0.90,59.97,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-190.91,74.0,M-SC,22.59,156960.0,7740.0,136445.0,-0.41,5.19,86.93,96.62,208.0,0.06,1.11,69.86,XY25,NTT,FINANCE
79,UJJIVANSFB,60.00,121.70,68.0,H-SC,15.24,140103.0,-2376.0,21898.0,-0.02,-1.67,15.63,13.70,163.0,-0.11,0.99,65.36,OX40N,NTT,BANKS
68,SONACOMS,806.63,-32.59,70.0,M-SC,9.59,83886.0,-17273.0,57277.0,0.27,-17.07,68.28,39.54,202.0,-0.30,0.59,18.29,AR,ATH,AUTO
5,ANGELONE,3033.00,15.82,68.0,X-SC,6.41,204680.0,13674.0,44027.0,-0.59,7.16,21.51,30.21,157.0,0.31,1.45,29.53,X40N,NTT,FINANCE
59,REPCOHOME,880.00,-56.04,69.0,H-SC,2.92,253967.0,-32538.0,289878.0,-0.33,-11.36,114.14,89.82,134.0,-0.11,1.80,34.37,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.43
1,25,43.81
2,50,74.73


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.38
LC    32.77
MC    22.84
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.31
X40      16.08
XY25     12.15
XR       11.28
X40N      9.33
AR        7.91
OX40N     7.63
X200      1.79
X5K       1.52
SR        1.09
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.58
X-LC    21.88
X-MC    17.06
M-SC    12.68
M-LC     5.19
H-LC     4.67
X-SC     4.64
H-MC     3.88
M-MC     1.60
L-SC     1.48
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.60,-4.57,38.70
IT,12.77,-16.54,79.37
FINANCE,12.14,-10.24,60.21
MISC,6.94,-17.15,82.06
BANKS,6.61,-6.89,63.12
PAINTS,5.63,-16.13,33.45
ELECTRICAL,5.32,-9.70,48.19
INSURANCE,3.90,-0.45,37.90
AUTO,2.81,-44.55,104.88


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3292129.0,22
XR,1316580.0,14
AR,1178164.0,9
X40,776991.0,12
X40N,612328.0,8
OX40N,568922.0,10
XY25,466300.0,8
SR,267510.0,2
X5K,102902.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3386424.0,25
M-SC,1216312.0,15
X-MC,1128420.0,13
X-LC,1048058.0,13
X-SC,526288.0,6
H-MC,332095.0,3
H-LC,278792.0,3
M-LC,274655.0,4
L-SC,252398.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1167188.0      6
           AR         846111.0      5
           XR         785379.0      7
M-SC       XY24       694293.0      6
X-MC       XY24       591929.0      4
X-LC       X40        449901.0      5
           XY24       306355.0      2
X-SC       X40N       284913.0      4
H-SC       SR         267510.0      2
           OX40N      263777.0      4
X-MC       X40        250176.0      6
X-LC       X40N       210393.0      3
H-LC       AR         207898.0      2
H-MC       XY24       177999.0      1
X-MC       XY25       169293.0      2
M-MC       XY24       167043.0      1
X-SC       XY24       164461.0      1
L-SC       XR         163197.0      2
M-SC       OX40N      147012.0      4
           XY25       136445.0      1
           AR         124155.0      2
X-MC       X40N       117022.0      1
M-SC       XR         114407.0      2
M-LC       XR         108795.0      1
           X5K        102902.0      1
L-SC       OX40N       89201.0      1
H-MC       XR          85164.0      1
X-LC       XY25        81409.0      3
X-SC       X40         76914.0      1
H-LC       X200        70894.0      1
H-MC       OX40N       68932.0      1
L-MC       XR          59638.0      1
H-SC       MH          56459.0      1
M-LC       XY25        40097.0      1
L-LC       XY25        39056.0      1
M-LC       XY24        22861.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
